In [1]:
import os
import urllib.request

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas import Series, DataFrame

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient

In [2]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://truenas.local:9000"
os.environ["MLFLOW_TRACKING_URI"] = "http://192.168.1.14:5000"

EXPERIMENT_NAME = "Weather Forecast Model Experiment"

In [3]:
client = MlflowClient()

In [5]:
reg_model = client.get_registered_model("KrasnodarWeatherForecastModel")

# Load registered model

In [10]:
model_uri = "models:/KrasnodarWeatherForecastModel/production"
loaded_model = mlflow.pyfunc.load_model(model_uri)

2023/07/11 14:46:18 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.2, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [26]:
sklearn_model = mlflow.sklearn.load_model(model_uri)
sklearn_model

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.03, max_depth=2,
                                            n_estimators=200))])

In [11]:
loaded_model.metadata

In [13]:
data = urllib.request.urlopen("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Russia%2C%20Krasnodar/today?unitGroup=metric&include=days&key=|||||||&contentType=csv")
df = pd.read_csv(data)
df

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Краснодар, Южный федеральный округ, Россия",2023-07-11,25,17,20.3,25,17,20.3,14.8,71.7,...,16.1,8,30,2023-07-11T04:47:47,2023-07-11T20:11:02,0.78,"Rain, Partially cloudy",Partly cloudy throughout the day with storms p...,rain,URKK


# Data preprocessing for prediction

In [22]:
df_for_pred = df.drop(['name', 'conditions', 'datetime', 'description', 'moonphase', 'precipprob', 'preciptype', 'snow', 'snowdepth', 'stations', 'sunrise', 'sunset', 'severerisk'], axis=1)
df_for_pred['windgust'] = df_for_pred['windgust'].fillna(0.0)
y_for_pred = df_for_pred['icon']
df_for_pred = df_for_pred.drop(['icon'], axis=1)
df_for_pred = df_for_pred.astype(np.float64)
df_for_pred['uvindex'] = df_for_pred['uvindex'].astype(np.int64)

print(f"X: {df_for_pred}\n\ny: {y_for_pred}")

X:    tempmax  tempmin  temp  feelslikemax  feelslikemin  feelslike   dew  \
0     25.0     17.0  20.3          25.0          17.0       20.3  14.8   

   humidity  precip  precipcover  windgust  windspeed  winddir  \
0      71.7     2.3        33.33      42.1       28.8    243.4   

   sealevelpressure  cloudcover  visibility  solarradiation  solarenergy  \
0            1011.3        45.7        13.0           185.6         16.1   

   uvindex  
0        8  

y: 0    rain
Name: icon, dtype: object


In [23]:
df_for_pred

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipcover,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex
0,25.0,17.0,20.3,25.0,17.0,20.3,14.8,71.7,2.3,33.33,42.1,28.8,243.4,1011.3,45.7,13.0,185.6,16.1,8


In [24]:
y_pred = loaded_model.predict(df_for_pred)
y_pred

array(['rain'], dtype=object)

In [27]:
sklearn_model.predict_proba(df_for_pred)

array([[2.96508713e-04, 3.07553626e-05, 3.27368911e-06, 4.60538337e-04,
        9.97836388e-01, 1.37005783e-03, 2.47796694e-06]])

# Test Clear-Day

In [2]:
data = urllib.request.urlopen("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Russia%2C%20Krasnodar/today?unitGroup=metric&include=days&key=||||||&contentType=csv")
df = pd.read_csv(data)
df

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Краснодар, Южный федеральный округ, Россия",2023-07-22,33.3,19,26.2,32.6,19,26.3,17,61.4,...,28.3,9,10,2023-07-22T04:57:58,2023-07-22T20:02:38,0.15,Clear,Clear conditions throughout the day.,clear-day,URKK


In [4]:
df = DataFrame(data=df.values.tolist(), columns=df.columns.tolist())
df

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Краснодар, Южный федеральный округ, Россия",2023-07-21,31,15,24,31.3,15,24.1,15.9,63.3,...,28.5,9,10,2023-07-21T04:56:57,2023-07-21T20:03:34,0.12,Clear,Clear conditions throughout the day.,clear-day,URKK


In [6]:
df_for_pred = df.drop(['name', 'conditions', 'datetime', 'description', 'moonphase', 'precipprob', 'preciptype', 'snow', 'snowdepth', 'stations', 'sunrise', 'sunset', 'severerisk'], axis=1)
df_for_pred['windgust'] = df_for_pred['windgust'].fillna(0.0)
y_for_pred = df_for_pred['icon']
df_for_pred = df_for_pred.drop(['icon'], axis=1)
df_for_pred = df_for_pred.astype(np.float64)
df_for_pred['uvindex'] = df_for_pred['uvindex'].astype(np.int64)

print(f"X: {df_for_pred}\n\ny: {y_for_pred}")

X:    tempmax  tempmin  temp  feelslikemax  feelslikemin  feelslike   dew  \
0     33.3     19.0  26.2          32.6          19.0       26.3  17.0   

   humidity  precip  precipcover  windgust  windspeed  winddir  \
0      61.4     0.0          0.0      19.1       10.8    292.0   

   sealevelpressure  cloudcover  visibility  solarradiation  solarenergy  \
0            1014.0        17.4        16.5           326.0         28.3   

   uvindex  
0        9  

y: 0    clear-day
Name: icon, dtype: object


In [32]:
y_pred = loaded_model.predict(df_for_pred)
y_pred

array(['clear-day'], dtype=object)

In [33]:
sklearn_model.predict_proba(df_for_pred)

array([[9.98273853e-01, 1.05622052e-06, 1.12509563e-07, 1.42664656e-03,
        2.51155911e-04, 4.70888873e-05, 8.73892207e-08]])

In [3]:
import urllib.request

data = urllib.request.urlopen("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Russia%2C%20Krasnodar/today?unitGroup=metric&include=days&key=|||||||&contentType=csv")

In [64]:
import time
import datetime

datetime.datetime.now().strftime("%d/%m/%Y|%H:%M:%S")

'19/07/2023|16:00:56'

In [6]:
df = pd.read_csv(data)
df

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Краснодар, Южный федеральный округ, Россия",2023-07-21,32.6,15,24.5,31.3,15,24.1,14.2,57.9,...,28.5,9,10,2023-07-21T04:56:57,2023-07-21T20:03:34,0.12,Clear,Clear conditions throughout the day.,clear-day,URKK


In [9]:
raw_data = df_for_pred.values.tolist()
# columns_data = df_for_pred.columns.tolist()
# columns_data
raw_data
# pd.DataFrame(raw_data, columns=columns_data)

[[33.3,
  19.0,
  26.2,
  32.6,
  19.0,
  26.3,
  17.0,
  61.4,
  0.0,
  0.0,
  19.1,
  10.8,
  292.0,
  1014.0,
  17.4,
  16.5,
  326.0,
  28.3,
  9.0]]

## Serving Test

In [20]:
import pandas as pd
import requests

json_data = df_for_pred.to_json(orient="records")
json_data = eval(json_data)[0]

In [21]:
json_data = {"dataframe_split": json_data}
json_data

{'dataframe_split': {'tempmax': 31.0,
  'tempmin': 15.0,
  'temp': 24.0,
  'feelslikemax': 31.3,
  'feelslikemin': 15.0,
  'feelslike': 24.1,
  'dew': 15.9,
  'humidity': 63.3,
  'precip': 0.0,
  'precipcover': 0.0,
  'windgust': 26.6,
  'windspeed': 21.6,
  'winddir': 9.7,
  'sealevelpressure': 1010.8,
  'cloudcover': 15.5,
  'visibility': 11.8,
  'solarradiation': 328.1,
  'solarenergy': 28.5,
  'uvindex': 9}}